## Import libraries
Import libraries to perform one hot coding and rescaling of features

In [34]:
## Import librariries
import sys, os
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump, load
import pickle
from typing import List, Tuple
from datetime import datetime
from dateutil import parser

## Import raw features
Import the raw feature file and inspect it.

In [35]:
# Read the raw feature CSV file into a pandas DataFrame
raw_feature_df = pd.read_csv("/Users/adityaponnada/Downloads/time_study_data/raw_features_v100.csv")
print(f"Raw feature DataFrame shape: {raw_feature_df.shape}")
raw_feature_df.head()

Raw feature DataFrame shape: (1087353, 23)


,participant_id,prompt_time_converted,outcome,is_weekend,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,...,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_1h,time_between_prompts,time_since_last_answered,completion_since_wake,completion_since_start
0,arrivejanitoruniformly@timestudy_com,2021-10-23 00:24:01,1,1,Late Night,NaN,NaN,Home,0,0.009990,...,200.983333,819.016667,38.425090,250,0.00,0.000000,0.000000,0.000000,0.00,0.00
1,arrivejanitoruniformly@timestudy_com,2021-10-23 00:37:00,1,1,Late Night,0.0,0.0,Home,0,0.010297,...,188.000000,832.000000,45.288044,250,0.00,0.000000,12.966667,0.000000,0.00,0.00
2,arrivejanitoruniformly@timestudy_com,2021-10-23 00:46:02,1,1,Late Night,NaN,NaN,Home,0,0.009733,...,178.966667,841.033333,56.353667,250,0.00,0.000000,8.000000,0.000000,0.00,0.00
3,arrivejanitoruniformly@timestudy_com,2021-10-23 00:54:02,1,1,Late Night,NaN,NaN,Home,0,0.009829,...,170.966667,849.033333,42.199854,250,0.00,0.000000,21.016667,0.000000,0.00,0.00
4,arrivejanitoruniformly@timestudy_com,2021-10-23 03:09:03,1,1,Late Night,NaN,NaN,Home,0,0.008112,...,35.950000,984.050000,61.503609,250,0.25,0.333333,18.033333,18.033333,0.25,0.25


In [38]:
test_pid = 'chewingslouchingfailing@timestudy_com'
df_user = raw_feature_df[raw_feature_df['participant_id'] == test_pid].copy()
df_user

,participant_id,prompt_time_converted,outcome,is_weekend,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,...,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_1h,time_between_prompts,time_since_last_answered,completion_since_wake,completion_since_start
117523,chewingslouchingfailing@timestudy_com,2021-10-23 07:12:06,0,1,Early Morning,NaN,NaN,Home,0,0.019691,...,1007.900000,12.100000,0.000000,340,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
117524,chewingslouchingfailing@timestudy_com,2021-10-23 07:36:46,0,1,Early Morning,0.0,0.0,Home,0,0.020342,...,983.233333,36.766667,0.000000,340,1.000000,1.000000,15.000000,15.000000,1.000000,1.000000
117525,chewingslouchingfailing@timestudy_com,2021-10-23 07:44:05,0,1,Early Morning,NaN,NaN,Home,0,0.020336,...,975.916667,44.083333,3.974274,340,1.000000,1.000000,7.983333,7.983333,1.000000,1.000000
117526,chewingslouchingfailing@timestudy_com,2021-10-23 07:54:44,1,1,Early Morning,NaN,NaN,Other,1,NaN,...,965.266667,54.733333,13.633029,340,1.000000,1.000000,20.016667,20.016667,1.000000,1.000000
117527,chewingslouchingfailing@timestudy_com,2021-10-23 08:17:26,0,1,Morning,NaN,NaN,Home,0,0.021143,...,942.566667,77.433333,158.352198,340,1.000000,1.000000,15.016667,15.016667,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130801,chewingslouchingfailing@timestudy_com,2020-11-25 20:46:04,1,0,Night,NaN,NaN,Home,1,0.007379,...,118.933333,826.066667,28.000090,8,0.830769,1.000000,8.550000,8.550000,0.830769,0.830769
130802,chewingslouchingfailing@timestudy_com,2020-11-25 20:54:02,1,0,Night,NaN,NaN,Home,1,0.007177,...,110.966667,834.033333,34.156058,8,0.850000,0.000000,512.016667,512.016667,0.000000,0.000000
130803,chewingslouchingfailing@timestudy_com,2020-11-25 21:19:02,1,0,Night,1.0,0.0,Home,1,0.007539,...,85.966667,859.033333,21.377373,8,0.833333,0.000000,12.550000,524.566667,0.000000,0.000000
130804,chewingslouchingfailing@timestudy_com,2020-11-25 21:27:01,1,0,Night,NaN,NaN,Home,1,0.007370,...,77.983333,867.016667,14.645125,8,0.816667,0.000000,7.433333,532.000000,0.000000,0.000000


In [20]:
# Observations per participant (placed immediately after raw_feature_df is created)
from IPython.display import display

def observations_per_participant(df, id_col='participant_id'):
    """Return participant-level observation counts as a DataFrame with columns [id_col, 'n_obs']"""
    if df is None:
        print('Provided DataFrame is None')
        return pd.DataFrame(columns=[id_col, 'n_obs'])
    if id_col not in df.columns:
        raise ValueError(f"id_col '{id_col}' not found in DataFrame")
    counts = df.groupby(id_col).size().reset_index(name='n_obs')
    # Sort by participant id for deterministic ordering
    counts = counts.sort_values(by=id_col, ascending=True).reset_index(drop=True)
    return counts

# Compute and display counts using the freshly-loaded `raw_feature_df`
obs_counts = observations_per_participant(raw_feature_df)
print(f"Participants: {obs_counts.shape[0]} | Total observations: {len(raw_feature_df)}")
display(obs_counts.head(50))

Participants: 100 | Total observations: 1087353


,participant_id,n_obs
0,arrivejanitoruniformly@timestudy_com,10845
1,atlanticchefhatchet@timestudy_com,13238
2,badlandwiltmuseum@timestudy_com,12257
3,bannisterhardwiredladle@timestudy_com,12757
4,bartenderradiatorapplied@timestudy_com,4692
5,beavertomatoupscale@timestudy_com,13099
6,bondingcoasterdirtiness@timestudy_com,13174
7,brinkaminounframed@timestudy_com,11267
8,catsupexploitmocker@timestudy_com,14639
9,certifiedembargobartender@timestudy_com,11555


In [21]:
# Min and max of days_in_study per participant
from IPython.display import display

def min_max_days_by_participant(df, id_col='participant_id', days_col='days_in_study'):
    """Return a DataFrame with columns [id_col, 'days_min', 'days_max'] for each participant."""
    if df is None:
        print('Provided DataFrame is None')
        return pd.DataFrame(columns=[id_col, 'days_min', 'days_max'])
    if id_col not in df.columns:
        raise ValueError(f"id_col '{id_col}' not found in DataFrame")
    if days_col not in df.columns:
        raise ValueError(f"days_col '{days_col}' not found in DataFrame")
    # Coerce to numeric, preserve NaNs
    days_numeric = pd.to_numeric(df[days_col], errors='coerce')
    tmp = df.copy()
    tmp[days_col] = days_numeric
    agg = tmp.groupby(id_col)[days_col].agg(['min', 'max']).reset_index().rename(columns={'min':'days_min', 'max':'days_max'})
    # Sort by participant id for deterministic output
    agg = agg.sort_values(by=id_col, ascending=True).reset_index(drop=True)
    return agg

# Compute and display the min/max table using the raw feature DataFrame
minmax_days = min_max_days_by_participant(raw_feature_df)
print(f"Participants: {minmax_days.shape[0]} | Total observations: {len(raw_feature_df)}")
display(minmax_days.head(100))

Participants: 100 | Total observations: 1087353


,participant_id,days_min,days_max
0,arrivejanitoruniformly@timestudy_com,0,350
1,atlanticchefhatchet@timestudy_com,0,352
2,badlandwiltmuseum@timestudy_com,0,289
3,bannisterhardwiredladle@timestudy_com,0,350
4,bartenderradiatorapplied@timestudy_com,0,277
...,...,...,...
95,vagabondnumerousflatterer@timestudy_com,0,326
96,whoeverrelightspookily@timestudy_com,0,199
97,wikipediaetchingcrystal@timestudy_com,0,348
98,wrigglecatalyststerility@timestudy_com,0,338


In [22]:
from IPython.display import display

def print_participant_head_tail(df, participant_id, id_col: str = 'participant_id', n: int = 5):
    """
    Simple utility: print the head and tail of `df` for the given participant_id.

    Parameters
    - df: pandas DataFrame containing participant rows
    - participant_id: value of the participant id to filter
    - id_col: name of the participant id column (default 'participant_id')
    - n: number of rows to show from head and tail (default 5)
    """
    if df is None:
        print('Provided DataFrame is None')
        return
    if id_col not in df.columns:
        print(f"id_col '{id_col}' not found in DataFrame columns")
        return
    sub = df[df[id_col] == participant_id]
    if sub.empty:
        print(f"No rows found for {id_col}={participant_id}")
        return
    print(f"--- HEAD ({n}) for participant {participant_id} ---")
    display(sub.head(n))
    print(f"--- TAIL ({n}) for participant {participant_id} ---")
    display(sub.tail(n))

# Example usage (uncomment and replace with a real id):
print_participant_head_tail(raw_feature_df, 'arrivejanitoruniformly@timestudy_com', n=5)


--- HEAD (5) for participant arrivejanitoruniformly@timestudy_com ---


,participant_id,prompt_time_converted,outcome,is_weekend,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,...,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_1h,time_between_prompts,time_since_last_answered,completion_since_wake,completion_since_start
0,arrivejanitoruniformly@timestudy_com,2021-10-23 00:24:01,1,1,Late Night,NaN,NaN,Home,0,0.009990,...,200.983333,819.016667,38.425090,250,0.00,0.000000,0.000000,0.000000,0.00,0.00
1,arrivejanitoruniformly@timestudy_com,2021-10-23 00:37:00,1,1,Late Night,0.0,0.0,Home,0,0.010297,...,188.000000,832.000000,45.288044,250,0.00,0.000000,12.966667,0.000000,0.00,0.00
2,arrivejanitoruniformly@timestudy_com,2021-10-23 00:46:02,1,1,Late Night,NaN,NaN,Home,0,0.009733,...,178.966667,841.033333,56.353667,250,0.00,0.000000,8.000000,0.000000,0.00,0.00
3,arrivejanitoruniformly@timestudy_com,2021-10-23 00:54:02,1,1,Late Night,NaN,NaN,Home,0,0.009829,...,170.966667,849.033333,42.199854,250,0.00,0.000000,21.016667,0.000000,0.00,0.00
4,arrivejanitoruniformly@timestudy_com,2021-10-23 03:09:03,1,1,Late Night,NaN,NaN,Home,0,0.008112,...,35.950000,984.050000,61.503609,250,0.25,0.333333,18.033333,18.033333,0.25,0.25


--- TAIL (5) for participant arrivejanitoruniformly@timestudy_com ---


,participant_id,prompt_time_converted,outcome,is_weekend,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,...,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_1h,time_between_prompts,time_since_last_answered,completion_since_wake,completion_since_start
10840,arrivejanitoruniformly@timestudy_com,2021-10-12 22:53:00,0,0,Night,0.0,1.0,Home,0,0.010586,...,127.000000,698.000000,0.000000,239,0.829457,0.829457,0.0,0.0,0.0,0.0
10841,arrivejanitoruniformly@timestudy_com,2021-10-12 23:25:01,0,0,Night,0.0,1.0,Home,0,0.010269,...,94.983333,730.016667,0.556412,239,0.830116,0.830116,0.0,0.0,0.0,0.0
10842,arrivejanitoruniformly@timestudy_com,2021-10-12 23:33:01,0,0,Night,0.0,1.0,Home,0,0.010807,...,86.983333,738.016667,0.631078,239,0.830769,0.830769,0.0,0.0,0.0,0.0
10843,arrivejanitoruniformly@timestudy_com,2021-10-12 23:43:01,0,0,Night,NaN,NaN,Home,0,0.010293,...,76.983333,748.016667,0.000000,239,0.831418,0.831418,0.0,0.0,0.0,0.0
10844,arrivejanitoruniformly@timestudy_com,2021-10-12 23:53:01,0,0,Night,NaN,NaN,Home,0,0.011239,...,66.983333,758.016667,0.000000,239,0.832061,0.832061,0.0,0.0,0.0,0.0


In [33]:
def missing_value_table(df):
    skip_cols = ['participant_id', 'prompt_time_converted', 'outcome']
    cols = [col for col in df.columns if col.lower() not in skip_cols]
    missing_percent = df[cols].isnull().mean() * 100
    # Also count empty strings as missing
    empty_percent = (df[cols] == '').mean() * 100
    total_missing_percent = missing_percent + empty_percent
    result = pd.DataFrame({
        'missing_%': total_missing_percent.round(2)
    }).sort_values('missing_%', ascending=False)
    return result
    
# Display missing value table
missing_value_table(raw_feature_df)

,missing_%
in_battery_saver_mode,52.06
charging_status,52.06
last_phone_usage,17.77
is_phone_locked,17.56
dist_from_home,17.05
mims_5min,5.33
time_of_day,0.27
closeness_to_sleep_time,0.19
wake_day_part,0.19
closeness_to_wake_time,0.19


In [24]:
def missing_data_by_participant(df):
    skip_cols = ['prompt_time_converted', 'outcome']
    cols = [col for col in df.columns if col.lower() not in skip_cols and col.lower() != 'participant_id']
    # Create a boolean DataFrame for missing values (NaN or empty string)
    missing_bool = df[cols].isnull() | (df[cols] == '')
    # Group by participant_id and calculate % missing for each column
    missing_percent = missing_bool.groupby(df['participant_id']).mean() * 100
    missing_percent = missing_percent.round(2)
    # Reset index to have participant_id as a column
    missing_percent = missing_percent.reset_index()
    return missing_percent

# Print % missing data for each participant and variable
missing_data_by_participant(raw_feature_df)

,participant_id,is_weekend,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,is_phone_locked,last_phone_usage,...,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_1h,time_between_prompts,time_since_last_answered,completion_since_wake,completion_since_start
0,arrivejanitoruniformly@timestudy_com,0.0,2.43,52.67,52.67,0.0,0.0,5.28,0.00,0.00,...,0.00,0.00,1.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,atlanticchefhatchet@timestudy_com,0.0,0.00,50.65,50.65,0.0,0.0,6.51,0.00,0.00,...,0.00,0.00,4.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,badlandwiltmuseum@timestudy_com,0.0,0.00,51.86,51.86,0.0,0.0,4.36,0.60,0.67,...,0.00,0.00,4.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bannisterhardwiredladle@timestudy_com,0.0,0.00,52.62,52.62,0.0,0.0,16.95,0.00,0.00,...,0.00,0.00,6.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,bartenderradiatorapplied@timestudy_com,0.0,0.00,69.27,69.27,0.0,0.0,51.92,100.00,100.00,...,1.32,1.32,26.45,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,vagabondnumerousflatterer@timestudy_com,0.0,0.00,51.35,51.35,0.0,0.0,55.70,22.91,25.51,...,0.54,0.54,7.91,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,whoeverrelightspookily@timestudy_com,0.0,0.00,62.86,62.86,0.0,0.0,16.42,4.76,4.76,...,0.00,0.00,34.13,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,wikipediaetchingcrystal@timestudy_com,0.0,0.00,51.45,51.45,0.0,0.0,10.92,0.00,0.00,...,0.00,0.00,1.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,wrigglecatalyststerility@timestudy_com,0.0,0.00,52.13,52.13,0.0,0.0,6.98,0.38,0.38,...,0.00,0.00,2.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## one-hot encoding
Convert the categorical variables into one-hot dummy codes

In [25]:
def one_hot_encode_features(df, columns):
    """
    One-hot encode specified categorical columns in the DataFrame.
    Returns a new DataFrame with one-hot encoded columns as 0/1 integers and original columns dropped.
    """
    df_encoded = df.copy()
    df_encoded = pd.get_dummies(df_encoded, columns=columns, prefix=columns, drop_first=False)
    # Ensure all new one-hot columns are int (0/1)
    for col in df_encoded.columns:
        if any(col.startswith(f'{c}_') for c in columns):
            df_encoded[col] = df_encoded[col].astype(int)
    return df_encoded

# Example usage:
categorical_vars = ['time_of_day', 'location_category', 'wake_day_part']
raw_feature_df_encoded = one_hot_encode_features(raw_feature_df, categorical_vars)
raw_feature_df_encoded.head()

,participant_id,prompt_time_converted,outcome,is_weekend,in_battery_saver_mode,charging_status,screen_on,dist_from_home,is_phone_locked,last_phone_usage,...,wake_day_part_24.0,wake_day_part_25.0,wake_day_part_26.0,wake_day_part_27.0,wake_day_part_28.0,wake_day_part_29.0,wake_day_part_30.0,wake_day_part_31.0,wake_day_part_32.0,wake_day_part_33.0
0,arrivejanitoruniformly@timestudy_com,2021-10-23 00:24:01,1,1,NaN,NaN,0,0.009990,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,arrivejanitoruniformly@timestudy_com,2021-10-23 00:37:00,1,1,0.0,0.0,0,0.010297,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,arrivejanitoruniformly@timestudy_com,2021-10-23 00:46:02,1,1,NaN,NaN,0,0.009733,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,arrivejanitoruniformly@timestudy_com,2021-10-23 00:54:02,1,1,NaN,NaN,0,0.009829,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,arrivejanitoruniformly@timestudy_com,2021-10-23 03:09:03,1,1,NaN,NaN,0,0.008112,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
## List the column names in the DataFrame
def list_column_names(df: pd.DataFrame) -> List[str]:
    """
    List all column names in the DataFrame.
    """
    return df.columns.tolist()

## Use the function to get column names
column_names = list_column_names(raw_feature_df_encoded)
print("Column names in the DataFrame:")
print(column_names)

Column names in the DataFrame:
['participant_id', 'prompt_time_converted', 'outcome', 'is_weekend', 'in_battery_saver_mode', 'charging_status', 'screen_on', 'dist_from_home', 'is_phone_locked', 'last_phone_usage', 'closeness_to_sleep_time', 'closeness_to_wake_time', 'mims_5min', 'days_in_study', 'completion_24h', 'completion_1h', 'time_between_prompts', 'time_since_last_answered', 'completion_since_wake', 'completion_since_start', 'time_of_day_Afternoon', 'time_of_day_Early Morning', 'time_of_day_Evening', 'time_of_day_Late Night', 'time_of_day_Morning', 'time_of_day_Night', 'location_category_Home', 'location_category_Other', 'location_category_School', 'location_category_Transit', 'location_category_Work', 'wake_day_part_0.0', 'wake_day_part_1.0', 'wake_day_part_2.0', 'wake_day_part_3.0', 'wake_day_part_4.0', 'wake_day_part_5.0', 'wake_day_part_6.0', 'wake_day_part_7.0', 'wake_day_part_8.0', 'wake_day_part_9.0', 'wake_day_part_10.0', 'wake_day_part_11.0', 'wake_day_part_12.0', 'wake_

## Normalize features [Skip this step]

In [27]:
from sklearn.preprocessing import MinMaxScaler

def min_max_scale_by_participant(df, columns, group_col='participant_id'):
    """
    Apply min-max scaling (0-1) to specified columns, grouped by participant_id.
    Returns a new DataFrame with scaled columns (original columns replaced).
    """
    df_scaled = df.copy()
    scaler = MinMaxScaler()
    # Apply scaling for each participant
    for pid, group in df.groupby(group_col):
        idx = group.index
        scaled_values = scaler.fit_transform(group[columns])
        df_scaled.loc[idx, columns] = scaled_values
    return df_scaled

# Example usage:
scale_columns = ['dist_from_home', 'last_phone_usage', 'closeness_to_sleep_time', 'closeness_to_wake_time', 
                 'mims_5min', 'time_between_prompts', 'time_since_last_answered']
raw_feature_df_scaled = min_max_scale_by_participant(raw_feature_df_encoded, scale_columns)
raw_feature_df_scaled[scale_columns + ['participant_id']].head()

/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/.venv/lib/python3.13/site-packages/sklearn/utils/_array_api.py:686: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/.venv/lib/python3.13/site-packages/sklearn/utils/_array_api.py:706: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/.venv/lib/python3.13/site-packages/sklearn/utils/_array_api.py:686: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/adityaponnada/Documents/codework/real_time_prompting/real_time_prompting/.venv/lib/python3.13/site-packages/sklearn/utils/_array_api.py:706: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/Users/adityaponnada/Documents/codework/real_tim

,dist_from_home,last_phone_usage,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,time_between_prompts,time_since_last_answered,participant_id
0,0.000002,0.0,0.187031,0.807168,0.150510,0.000000,0.000000,arrivejanitoruniformly@timestudy_com
1,0.000002,0.0,0.180555,0.814221,0.177391,0.000856,0.000000,arrivejanitoruniformly@timestudy_com
2,0.000002,0.0,0.176050,0.819127,0.220735,0.000528,0.000000,arrivejanitoruniformly@timestudy_com
3,0.000002,0.0,0.172060,0.823473,0.165295,0.001388,0.000000,arrivejanitoruniformly@timestudy_com
4,0.000002,0.0,0.104721,0.896812,0.240907,0.001191,0.001191,arrivejanitoruniformly@timestudy_com


## FixedMax scaling for days in study

In [30]:
# Display head and tail for participant_id, prompt_time_converted, and days_in_study from the encoded DataFrame

from IPython.display import display
cols = ['participant_id', 'prompt_time_converted', 'days_in_study']
if 'raw_feature_df_encoded' in globals():
    df = raw_feature_df_encoded
    missing = [c for c in cols if c not in df.columns]
    if missing:
        print(f'Missing columns in raw_feature_df_encoded: {missing}')
    else:
        print('--- HEAD (participant_id, days_in_study) ---')
        display(df[cols].head())
        print('--- TAIL (participant_id, days_in_study) ---')
        display(df[cols].tail())
else:
    print("raw_feature_df_encoded not found in the notebook namespace. Run the one-hot encoding cell first.")

--- HEAD (participant_id, days_in_study) ---


,participant_id,prompt_time_converted,days_in_study
0,arrivejanitoruniformly@timestudy_com,2021-10-23 00:24:01,250
1,arrivejanitoruniformly@timestudy_com,2021-10-23 00:37:00,250
2,arrivejanitoruniformly@timestudy_com,2021-10-23 00:46:02,250
3,arrivejanitoruniformly@timestudy_com,2021-10-23 00:54:02,250
4,arrivejanitoruniformly@timestudy_com,2021-10-23 03:09:03,250


--- TAIL (participant_id, days_in_study) ---


,participant_id,prompt_time_converted,days_in_study
1087348,yearlingfiberspotty@timestudy_com,2021-10-12 21:44:03,95
1087349,yearlingfiberspotty@timestudy_com,2021-10-12 21:54:01,95
1087350,yearlingfiberspotty@timestudy_com,2021-10-12 22:30:01,95
1087351,yearlingfiberspotty@timestudy_com,2021-10-12 22:38:04,95
1087352,yearlingfiberspotty@timestudy_com,2021-10-12 22:46:01,95
